In [63]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [64]:
train_path = 'drive/My Drive/lfw_dataset/people' #input dataset

# Data Preparation 

1. **data split & augmentation**

In [65]:
train_datagen = ImageDataGenerator(validation_split=0.2) #split validation

2. **data flow**

In [66]:
train_generator = train_datagen.flow_from_directory(train_path, 
                                                    target_size=(224,224),
                                                    subset='training'
                                                    )

val_generator = train_datagen.flow_from_directory(train_path, # same directory as training data
                                                  target_size=(224,224),
                                                  subset='validation' # set as validation data
                                                  )

Found 400 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


In [67]:
# Label
classes = train_generator.class_indices

label_map = np.array(list(classes.items()))

label = label_map[:,0].tolist()
map = label_map[:,1].tolist()

print(label_map)
print('\n')
print(label)
print('\n')
print(map)

[['Ariel_Sharon' '0']
 ['Colin_Powell' '1']
 ['Donald_Rumsfeld' '2']
 ['George_W_Bush' '3']
 ['Gerhard_Schroeder' '4']
 ['Hugo_Chavez' '5']
 ['Jean_Chretien' '6']
 ['John_Ashcroft' '7']
 ['Junichiro_Koizumi' '8']
 ['Tony_Blair' '9']]


['Ariel_Sharon', 'Colin_Powell', 'Donald_Rumsfeld', 'George_W_Bush', 'Gerhard_Schroeder', 'Hugo_Chavez', 'Jean_Chretien', 'John_Ashcroft', 'Junichiro_Koizumi', 'Tony_Blair']


['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


3. **Global variable for training**

In [68]:
num_of_train_samples = train_generator.n
num_of_val_samples = val_generator.n
n_classes = val_generator.num_classes
batch_size = 32
epochs = 50

checkpoint_filepath = 'drive/My Drive/saved_model/my_model.h5'

# Model

In [69]:
# reset model
model = None
hist = None
opt = None

In [70]:
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Dropout

def my_model():
  chanDim = -1
  model = Sequential()
  model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(9,9),padding="same", activation="relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=128, kernel_size=(7,7), padding="same", activation="relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  #model.add(Dropout(rate=0.25))

  model.add(Conv2D(filters=256, kernel_size=(5,5), padding="same", activation="relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=512, kernel_size=(5,5), padding="same", activation="relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  #model.add(Dropout(rate=0.25))

  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

  model.add(Flatten())
  model.add(Dense(units=4096,activation="relu"))
  model.add(BatchNormalization())
  model.add(Dropout(rate=0.25))
  model.add(Dense(units=1024,activation="relu"))
  model.add(BatchNormalization())
  model.add(Dropout(rate=0.25))
  model.add(Dense(units=256,activation="relu"))
  model.add(BatchNormalization())
  model.add(Dropout(rate=0.25))
  model.add(Dense(units=64,activation="relu"))
  model.add(BatchNormalization())
  model.add(Dropout(rate=0.25))
  model.add(Dense(units=8,activation="relu"))
  model.add(BatchNormalization())
  model.add(Dropout(rate=0.25))
  model.add(Dense(units=n_classes, activation="softmax"))

  opt = Adam(learning_rate=0.0005,
             beta_1=0.8,
             beta_2=0.9,)
  
  model.compile(optimizer=opt, 
                loss=keras.losses.categorical_crossentropy, 
                metrics=['accuracy'])  

  return model


model = my_model()

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 224, 224, 64)      15616     
_________________________________________________________________
batch_normalization_31 (Batc (None, 224, 224, 64)      256       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 112, 112, 128)     401536    
_________________________________________________________________
batch_normalization_32 (Batc (None, 112, 112, 128)     512       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 56, 56, 256)      

# Train

In [71]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(mymodel_path, 
                             verbose=1,
                             monitor='val_accuracy',
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto')

hist = model.fit(train_generator,
                 steps_per_epoch=num_of_train_samples // batch_size,
                 epochs=epochs,
                 validation_data=val_generator,
                 validation_steps=num_of_val_samples // batch_size,
                 callbacks=[checkpoint])

Epoch 1/50
12/12 [==============================] - 9s 738ms/step - loss: 2.6206 - accuracy: 0.1196 - val_loss: 365.5289 - val_accuracy: 0.1042

Epoch 00001: val_accuracy improved from -inf to 0.10417, saving model to drive/My Drive/saved_model/my_model.h5
Epoch 2/50
12/12 [==============================] - 7s 559ms/step - loss: 2.3498 - accuracy: 0.1875 - val_loss: 208.2320 - val_accuracy: 0.0735

Epoch 00002: val_accuracy did not improve from 0.10417
Epoch 3/50
12/12 [==============================] - 6s 528ms/step - loss: 2.1874 - accuracy: 0.2174 - val_loss: 148.6240 - val_accuracy: 0.1176

Epoch 00003: val_accuracy improved from 0.10417 to 0.11765, saving model to drive/My Drive/saved_model/my_model.h5
Epoch 4/50
12/12 [==============================] - 6s 519ms/step - loss: 2.1941 - accuracy: 0.2554 - val_loss: 35.6730 - val_accuracy: 0.1912

Epoch 00004: val_accuracy improved from 0.11765 to 0.19118, saving model to drive/My Drive/saved_model/my_model.h5
Epoch 5/50
12/12 [======

# Prediction

In [73]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

saved_model = 'drive/My Drive/saved_model/my_model_val_acc80882.h5'
load_model = load_model(saved_model)

Y_pred = load_model.predict_generator(val_generator, num_of_val_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix \n')
print(confusion_matrix(val_generator.classes, y_pred))
print('\n\n accuracy score \n')
print(accuracy_score(val_generator.classes, y_pred))
print('\n \n Classification Report \n')

target_names = label

print(classification_report(val_generator.classes, y_pred, target_names=target_names))

Confusion Matrix 

[[1 2 1 3 1 1 0 0 1 0]
 [1 1 0 0 2 0 2 0 2 2]
 [0 1 3 1 0 1 2 1 1 0]
 [0 0 2 0 3 0 2 1 1 1]
 [1 0 2 0 2 0 0 1 3 1]
 [3 0 2 0 2 0 0 1 2 0]
 [0 0 1 0 1 1 1 3 1 2]
 [2 0 1 1 2 2 1 1 0 0]
 [2 1 1 0 1 0 2 0 0 3]
 [0 4 0 0 0 1 1 2 1 1]]


 accuracy score 

0.1

 
 Classification Report 

                   precision    recall  f1-score   support

     Ariel_Sharon       0.10      0.10      0.10        10
     Colin_Powell       0.11      0.10      0.11        10
  Donald_Rumsfeld       0.23      0.30      0.26        10
    George_W_Bush       0.00      0.00      0.00        10
Gerhard_Schroeder       0.14      0.20      0.17        10
      Hugo_Chavez       0.00      0.00      0.00        10
    Jean_Chretien       0.09      0.10      0.10        10
    John_Ashcroft       0.10      0.10      0.10        10
Junichiro_Koizumi       0.00      0.00      0.00        10
       Tony_Blair       0.10      0.10      0.10        10

         accuracy                           0.1